# Tweepy Handles Scraper

## INIT

In [1]:
import tweepy
import pandas as pd
import numpy as np

In [2]:
# assign the values accordingly 
consumer_key = "T1CPVPyMJiTQeHbM8wU3j4yUX" 
consumer_secret = "xlfVLzMPtAAjndIzeKDYhgtMYjdAOn1FkQe3q3NSBk9QTnxAEt" 
access_token = "3097042092-5ZHpgdeafTg9I3A6v15N3kbL5mHjV2r9TuaSrGD" 
access_token_secret = "a5Txr3k8vEN7G1U3wvVPdv8eh3p8hlWTAedLPy2WyGEfl"

In [3]:
# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
# set access to user's access key and access secret  
auth.set_access_token(access_token, access_token_secret) 
  
# calling the api  
api = tweepy.API(auth, wait_on_rate_limit=True)

In [6]:
df=pd.read_excel('../input/SnP_500_CEO_Names.xlsx')
df.dropna(inplace=True)
df.drop_duplicates(subset=['CEO Name', 'Company'], inplace=True)
#df['ceos']= df[['CEO Name', 'Company']].apply(lambda x: ' '.join(x), axis=1)
#df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.shape

(1647, 5)

In [7]:
df.columns

Index(['CEO Name', 'Company', 'GVKEY', 'EXECID', 'YEAR'], dtype='object')

In [8]:
tdf=pd.DataFrame()
ceos=[]
companies=[]
years=[]
handles=[]
descriptions=[]
verified=[]

for i in range(len(df)):
    users = api.search_users(df.iloc[i]['CEO Name'])
    lst=[]
    for user in users:
        lst.append(user.screen_name)
    for j in lst:
        result=api.get_user(j).verified
        if result==True:
            ceos.append(df.iloc[i]['CEO Name'])
            companies.append(df.iloc[i]['Company'])
            years.append(df.iloc[i]['YEAR'])
            handles.append(j)
            descriptions.append(api.get_user(j).description)
            verified.append(result)
        elif(('ceo' in api.get_user(j).description.lower()) and int(api.get_user(j).followers_count)>100):
            ceos.append(df.iloc[i]['CEO Name'])
            companies.append(df.iloc[i]['Company'])
            years.append(df.iloc[i]['YEAR'])
            handles.append(j)
            descriptions.append(api.get_user(j).description)
            verified.append(result)
        else:
            ceos.append(df.iloc[i]['CEO Name'])
            companies.append(df.iloc[i]['Company'])
            years.append(df.iloc[i]['YEAR'])
            handles.append(np.NaN)
            descriptions.append(np.NaN)
            verified.append(np.NaN)

In [9]:
columns = ['CEO','Company', 'YEAR', 'Handle', 'Twitter_Descr', 'verified']
tdf = pd.DataFrame(list(zip(ceos, companies, years, handles, descriptions, verified)), columns =columns) 
tdf.dropna(inplace=True)
tdf= tdf[tdf['verified']!=False]
tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 12 to 3950
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CEO            99 non-null     object
 1   Company        99 non-null     object
 2   YEAR           99 non-null     int64 
 3   Handle         99 non-null     object
 4   Twitter_Descr  99 non-null     object
 5   verified       99 non-null     object
dtypes: int64(1), object(5)
memory usage: 5.4+ KB


In [10]:
tdf.to_csv("../output/handles_tweepy_lookup.csv", index=False)